In [58]:
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import serialization, hashes
import base64

In [60]:
#문제 1-5.1
# private_key.pem 파일 경로
private_key_path = 'key.pem'  

#암호화 데이터 담는 변수
encrypted_data_b64 = "vVx4VTMrbXz9TPGaJc3OnvciWMWvpxTmRp0xg+oyo0gp+vI1Mb9Ix4jz0P8p6qFPdASUzm3pc/P7qelxdkJ1dXssS40hOR/38x5pHjd1+5uzeID4JukDGxbIfaAr0FIjjnFa1q0qsRAb0+FT/JxlZ1E409UxMvQaUXEDa4wP+U3eP045/yTuWuYjujY0Da4pU5QGzUzSzryvetRU8ondRrMSg1d4iPlj5UPhwHvTJuiRsjb/jVogYGexrCZjF+TL7jh+fKTpz1dFd5K5SYq2G+NjtlDcnrD9r/E1a3HEOn0JTTdwIDwWMi2/rvNaXGla+qtFAc0K0ry5MLhMlKvbgg=="

# 개인키 불러오기
with open(private_key_path, 'rb') as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password=None,
    )

# 암호화된 데이터 base64에서 디코딩
encrypted_data = base64.b64decode(encrypted_data_b64)

# RSA 비밀키를 사용하여 데이터 복호화
decrypted_data = private_key.decrypt(
    encrypted_data,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# 결과 출력
print("복호화된 메시지:", decrypted_data.decode('utf-8'))

복호화된 메시지: 40대 이상 여성


In [ ]:
#문제 1-5.2

# 비대칭 키 암호화 방식
# 두 개의 키인 공개 키(Public Key)와 개인 키(Private Key)를 사용해 데이터를 암호화하고 복호화하는 방식

# 암호화는 공개키로만 가능 복호화는 개인키로만 가능
# 반대로 하면 반대도 가능

# 데이터를 보내는 송신자가 공개 키를 통해 암호화
# 수신자의 개인키 아니면 복호화 불가능

# 하지만 개인키가 긴 단점이 존재함

# ECC(타원곡선암호화)

# 256비트 키는 RSA 3072비트 키와 유사한 보안성
# 비트가 줄어드니까, 키 보내는 속도가 빠름


# ECC (특히 256비트로도 높은 보안성 제공)
# 디피-헬만 (특히 ECDH)
# 엘가말
# 라빈
# RSA (긴 키 필요)
# 순으로 짧은 개인키 특성을 지님

# RSA: RSA의 보안은 큰 소수를 인수분해하는 데 걸리는 시간에 기반. 더 큰 숫자를 사용할수록 보안성은 높아지지만, 키 길이도 길어짐.
# ECC: 타원 곡선은 짧은 키에서도 안전한 보안성을 제공하기 때문에, ECC는 RSA와 비교해 키 길이 대비 효율성이 매우 높음. ECC는 복잡한 수학적 문제를 활용하므로, 비슷한 수준의 보안성을 위해 더 짧은 키를 사용해도 충분

In [42]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

In [72]:
#문제 1-5.3
# ECC 개인 키와 공개 키 생성
# SECP256R1은 타원 곡선 표준화 알고리즘의 대표값으로 높은 보안성 제공함
private_key = ec.generate_private_key(ec.SECP256R1())
public_key = private_key.public_key()

# 상대방의 공개 키 (일반적으로는 수신자의 공개 키를 사용해야 함)
# (상대방의 키가 있다고 가정했습니다.)
peer_private_key = ec.generate_private_key(ec.SECP256R1())
peer_public_key = peer_private_key.public_key()

# ECC를 통한 공유 비밀 키 생성 
shared_key = private_key.exchange(ec.ECDH(), peer_public_key)

# AES 대칭 키 파생 (HKDF를 사용하여 공유 키를 AES 키로 변환)
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data'
).derive(shared_key)

# 암호화할 메시지 설정
message = "신규 고객 수".encode('utf-8')

# AES로 메시지 암호화
iv = os.urandom(16)  # 초기화 벡터 생성
cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv))
encryptor = cipher.encryptor()
ciphertext = encryptor.update(message) + encryptor.finalize()

# 암호화된 데이터와 IV를 Base64로 인코딩하여 출력
ciphertext_base64 = base64.b64encode(iv + ciphertext)
print("암호화된 메시지(Base64):", ciphertext_base64.decode('utf-8'))

# 복호화 과정
# AES 대칭 키로 메시지 복호화
iv = base64.b64decode(ciphertext_base64)[:16]
ciphertext = base64.b64decode(ciphertext_base64)[16:]
cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv))
decryptor = cipher.decryptor()
decrypted_message = decryptor.update(ciphertext) + decryptor.finalize()

# 복호화된 메시지 출력
print("복호화된 메시지:", decrypted_message.decode('utf-8'))


암호화된 메시지(Base64): PFWj81+ZGoH4p5s3Gxf63YR+KprEC2X7IdOdziFciht1
복호화된 메시지: 신규 고객 수
